In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
LABELS = [True, False]
# True = home win, False = away win
LABEL_COLUMN = "WINorLOSS"

TEAM_DICT = {
             "MIL" : "Milwaukee Bucks",
             "MIA" : "Miami Heat",
             "BOS" : "Boston Celtics",
             "TOR" : "Toronto Raptors",
             "PHI" : "Philidelphia 76ers",
             "IND" : "Indiana Pacers",
             "BRK" : "Brooklyn Nets",
             "ORL" : "Orlando Magic",
             "CHO" : "Charlotte Hornets",
             "CHI" : "Chicago Bulls",
             "DET" : "Detroit Pistons",
             "WAS" : "Washington Wizards",
             "CLE" : "Cleveland Cavaliers",
             "NYK" : "New York Knicks",
             "ATL" : "Atlanta Hawks",
             "LAL" : "Los Angeles Lakers",
             "DEN" : "Denver Nuggets",
             "HOU" : "Houston Rockets",
             "LAC" : "Los Angeles Clippers",
             "DAL" : "Dallas Mavericks",
             "UTA" : "Utah Jazz",
             "OKC" : "Oklahoma City Thunder",
             "POR" : "Portland Trail Blazers",
             "SAS" : "San Antonio Spurs",
             "SAC" : "Sacramento Kings",
             "PHO" : "Phoenix Suns",
             "MEM" : "Memphis Grizzlies",
             "MIN" : "Minnesota Timberwolves",
             "NOP" : "New Orleans Pelicans",
             "GSW" : "Golden State Warriors"
            }

BOOL_DICT = {1 : True, 0 : False}

In [8]:
data2 = pd.read_csv("../data/games/nba-games-results-2014-2018.csv",index_col=1)

# removing and cleaning data
data2 = data2.drop(columns=['Unnamed: 0','Game'])

data2['Home'] = pd.Categorical(data2['Home'])
data2['Home'] = data2.Home.cat.codes
data2['WINorLOSS'] = pd.Categorical(data2['WINorLOSS'])
data2['WINorLOSS'] = data2.WINorLOSS.cat.codes
data2.index = data2.index.map(TEAM_DICT)
data2['Opponent'] = data2['Opponent'].map(TEAM_DICT)
#data2.drop(columns=[''])

data3 = pd.read_csv("../data/games/nba-games-2019-oct-nov-dec.txt", index_col=4)
data3 = data3.drop(columns=['Start (ET)','Unnamed: 6','Unnamed: 7', 'Attend.', 'Notes'])

data4 = pd.read_csv("../data/league-stats/team-per-game.txt", index_col=1)
data4 = data4.drop(columns=['Rk', 'G', 'MP'])

data2015 = pd.read_csv("../data/league-stats/team-per-game-14-15.txt",index_col=1)
data2015 = data2015.drop(columns=['Rk','G', 'MP'])

data2016 = pd.read_csv("../data/league-stats/team-per-game-15-16.txt",index_col=1)
data2016 = data2016.drop(columns=['Rk','G', 'MP'])

data2017 = pd.read_csv("../data/league-stats/team-per-game-16-17.txt",index_col=1)
data2017 = data2017.drop(columns=['Rk','G', 'MP'])

data2018 = pd.read_csv("../data/league-stats/team-per-game-17-18.txt",index_col=1)
data2018 = data2018.drop(columns=['Rk','G', 'MP'])

dataOppStats2020 = pd.read_csv("../data/league-stats/opponent-stats.txt", index_col=1)
dataOppStats2020 = dataOppStats2020.drop(columns=['Rk', 'G', 'MP'])

#data4['FG'] = data4['FG'] / data4['FG'].max()
#data4.iloc[:,0]
x,y = data4.shape


#standardizing data
#for i in range(y):
#    data4.iloc[:,i] = data4.iloc[:,i] / data4.iloc[:,i].max()

data3["HomeWin"] = data3["PTS"] > data3["PTS.1"]
data3 = data3.drop(columns=["PTS", "PTS.1"])

w,z = data2.shape

data2 = data2[data2["Home"] == 1]
data2 = data2.drop(columns=['Home'])
data2 = data2.rename(columns = {"Opponent" : "Visitor/Neutral",
                                "WINorLOSS" : "HomeWin"})
data2 = data2.drop(columns = data2.columns[3:])
data2["HomeWin"] = data2["HomeWin"].map(BOOL_DICT)

dataOppStats2020.loc['Milwaukee Bucks']




#input for prediction

#season stats (past 5 games?), opponent, home/away

#input: team stats (weighted average), opp stats (weighted average), home/away, rest days

FG      38.700
FGA     93.000
FG%      0.417
3P      14.200
3PA     38.400
3P%      0.369
2P      24.600
2PA     54.600
2P%      0.450
FT      16.300
FTA     21.300
FT%      0.764
ORB      9.800
DRB     36.100
TRB     45.900
AST     24.200
STL      6.600
BLK      4.400
TOV     15.300
PF      22.100
PTS    107.900
Name: Milwaukee Bucks, dtype: float64

In [10]:
d5 = data3.join(data4,lsuffix="_d3", rsuffix="_d4")
d6 = d5.join(data4, on='Visitor/Neutral', lsuffix="_home",rsuffix="_away")
d7 = d6.join(dataOppStats2020)
d8 = d7.join(dataOppStats2020, on='Visitor/Neutral', lsuffix='_homeOppStat', rsuffix='_awayOppStats')
# has teams, date, winner, all relevant stats: need to add rest days, recency bias, any other stats I may want (injury?)
row, col = d8.shape
#need to use later for ^^^^
d8 = d8.drop(columns=['Date'])



d8 = d8.sample(frac=1)
a = int(row * .2)
train_data = d8.iloc[0:a]
test_data = d8.iloc[a:row]

train_labels = train_data["HomeWin"]
train_labels = train_labels.to_numpy(dtype=bool)
train_data = train_data.drop(columns=["HomeWin"])

test_labels = test_data["HomeWin"]
test_labels = test_labels.to_numpy(dtype=bool)
test_data = test_data.drop(columns=["HomeWin"])

test_data = test_data.drop(columns=['Visitor/Neutral'])
train_data = train_data.drop(columns=['Visitor/Neutral'])

row_t, col_t = train_data.shape

train_data = train_data.values
test_data = test_data.values

train_data[0]



array([ 41.   ,  91.9  ,   0.446,  13.9  ,  38.5  ,   0.361,  27.1  ,
        53.5  ,   0.507,  16.   ,  21.9  ,   0.73 ,  10.6  ,  34.4  ,
        45.   ,  24.7  ,   7.5  ,   4.9  ,  16.   ,  22.6  , 111.9  ,
        38.1  ,  88.4  ,   0.431,  10.2  ,  30.8  ,   0.333,  27.9  ,
        57.7  ,   0.484,  17.   ,  21.8  ,   0.78 ,  10.5  ,  34.   ,
        44.4  ,  22.7  ,   8.2  ,   6.4  ,  13.   ,  18.1  , 103.5  ,
        42.5  ,  90.3  ,   0.471,  12.9  ,  34.8  ,   0.37 ,  29.7  ,
        55.5  ,   0.535,  20.6  ,  25.9  ,   0.795,   9.8  ,  36.8  ,
        46.5  ,  25.1  ,   7.9  ,   4.6  ,  15.2  ,  20.3  , 118.5  ,
        39.5  ,  86.2  ,   0.458,  12.2  ,  33.   ,   0.37 ,  27.3  ,
        53.2  ,   0.513,  14.   ,  18.8  ,   0.747,   9.4  ,  36.3  ,
        45.8  ,  23.7  ,   6.9  ,   4.6  ,  14.7  ,  19.5  , 105.3  ])

In [12]:
# creating the model
model = keras.Sequential([
    keras.layers.Input(col_t),
    keras.layers.Dense(256, activation='tanh'),
    keras.layers.Dense(128, activation='tanh'),
    keras.layers.Dense(2, activation='softmax')
])
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [13]:
model.fit(train_data, train_labels, epochs=5)

Train on 90 samples
Epoch 1/5
90/90 [==============================] - 1s 12ms/sample - loss: 0.9585 - accuracy: 0.4778
Epoch 2/5
90/90 [==============================] - 0s 142us/sample - loss: 0.6681 - accuracy: 0.5889
Epoch 3/5
90/90 [==============================] - 0s 163us/sample - loss: 0.6968 - accuracy: 0.5667
Epoch 4/5
90/90 [==============================] - 0s 185us/sample - loss: 0.6668 - accuracy: 0.6444
Epoch 5/5
90/90 [==============================] - 0s 172us/sample - loss: 0.6579 - accuracy: 0.7111


In [14]:
model.evaluate(test_data,test_labels)

364/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[0.6656037103998792, 0.5714286]

In [33]:
#predict = model.predict()
test = [test_data[0]]
test_data


array([[ 39.8  ,  88.2  ,   0.452, ...,  15.2  ,  20.3  , 118.5  ],
       [ 38.7  ,  87.8  ,   0.441, ...,  18.6  ,  20.6  , 107.4  ],
       [ 40.7  ,  89.9  ,   0.453, ...,  15.4  ,  22.9  , 107.4  ],
       ...,
       [ 42.6  ,  88.   ,   0.484, ...,  16.   ,  20.8  , 121.8  ],
       [ 38.1  ,  88.6  ,   0.43 , ...,  15.1  ,  20.8  , 110.5  ],
       [ 38.7  ,  87.8  ,   0.441, ...,  15.3  ,  22.1  , 107.9  ]])